In [2]:
import os
import numpy as np
import pandas as pd
import swifter
from sklearn.model_selection import train_test_split
import utils
import catboost

['/home/ihar/catboost/catboost/python-package/catboost/./_catboost.so']


In [63]:
import os
from itertools import repeat
import numpy as np
import pandas as pd
    

SIMPLE_FEATURE_COLUMNS_FLOAT = ['ncl[0]', 'ncl[1]', 'ncl[2]', 'ncl[3]', 'avg_cs[0]',
       'avg_cs[1]', 'avg_cs[2]', 'avg_cs[3]',
       'MatchedHit_X[0]', 'MatchedHit_X[1]', 'MatchedHit_X[2]',
       'MatchedHit_X[3]', 'MatchedHit_Y[0]', 'MatchedHit_Y[1]',
       'MatchedHit_Y[2]', 'MatchedHit_Y[3]', 'MatchedHit_Z[0]',
       'MatchedHit_Z[1]', 'MatchedHit_Z[2]', 'MatchedHit_Z[3]',
       'MatchedHit_DX[0]', 'MatchedHit_DX[1]', 'MatchedHit_DX[2]',
       'MatchedHit_DX[3]', 'MatchedHit_DY[0]', 'MatchedHit_DY[1]',
       'MatchedHit_DY[2]', 'MatchedHit_DY[3]', 'MatchedHit_DZ[0]',
       'MatchedHit_DZ[1]', 'MatchedHit_DZ[2]', 'MatchedHit_DZ[3]',
       'MatchedHit_T[0]', 'MatchedHit_T[1]', 'MatchedHit_T[2]',
       'MatchedHit_T[3]', 'MatchedHit_DT[0]', 'MatchedHit_DT[1]',
       'MatchedHit_DT[2]', 'MatchedHit_DT[3]', 'Lextra_X[0]', 'Lextra_X[1]',
       'Lextra_X[2]', 'Lextra_X[3]', 'Lextra_Y[0]', 'Lextra_Y[1]',
       'Lextra_Y[2]', 'Lextra_Y[3]', 'NShared', 'Mextra_DX2[0]',
       'Mextra_DX2[1]', 'Mextra_DX2[2]', 'Mextra_DX2[3]', 'Mextra_DY2[0]',
       'Mextra_DY2[1]', 'Mextra_DY2[2]', 'Mextra_DY2[3]', 'FOI_hits_N', 'PT', 'P',
                         ]

TRAIN_COLUMNS = ["label", "weight", "particle_type", 'sWeight', 'kinWeight']

FOI_COLUMNS = ["FOI_hits_X", "FOI_hits_Y", "FOI_hits_T",
               "FOI_hits_Z", "FOI_hits_DX", "FOI_hits_DY", "FOI_hits_S"]



SIMPLE_FEATURE_COLUMNS = ['ncl[0]', 'ncl[1]', 'ncl[2]', 'ncl[3]', 'avg_cs[0]',
       'avg_cs[1]', 'avg_cs[2]', 'avg_cs[3]', 'ndof', 
                          'MatchedHit_TYPE[0]',
       'MatchedHit_TYPE[1]', 'MatchedHit_TYPE[2]', 'MatchedHit_TYPE[3]',
       'MatchedHit_X[0]', 'MatchedHit_X[1]', 'MatchedHit_X[2]',
       'MatchedHit_X[3]', 'MatchedHit_Y[0]', 'MatchedHit_Y[1]',
       'MatchedHit_Y[2]', 'MatchedHit_Y[3]', 'MatchedHit_Z[0]',
       'MatchedHit_Z[1]', 'MatchedHit_Z[2]', 'MatchedHit_Z[3]',
       'MatchedHit_DX[0]', 'MatchedHit_DX[1]', 'MatchedHit_DX[2]',
       'MatchedHit_DX[3]', 'MatchedHit_DY[0]', 'MatchedHit_DY[1]',
       'MatchedHit_DY[2]', 'MatchedHit_DY[3]', 'MatchedHit_DZ[0]',
       'MatchedHit_DZ[1]', 'MatchedHit_DZ[2]', 'MatchedHit_DZ[3]',
       'MatchedHit_T[0]', 'MatchedHit_T[1]', 'MatchedHit_T[2]',
       'MatchedHit_T[3]', 'MatchedHit_DT[0]', 'MatchedHit_DT[1]',
       'MatchedHit_DT[2]', 'MatchedHit_DT[3]', 'Lextra_X[0]', 'Lextra_X[1]',
       'Lextra_X[2]', 'Lextra_X[3]', 'Lextra_Y[0]', 'Lextra_Y[1]',
       'Lextra_Y[2]', 'Lextra_Y[3]', 'NShared', 'Mextra_DX2[0]',
       'Mextra_DX2[1]', 'Mextra_DX2[2]', 'Mextra_DX2[3]', 'Mextra_DY2[0]',
       'Mextra_DY2[1]', 'Mextra_DY2[2]', 'Mextra_DY2[3]',
                          'FOI_hits_N', 'PT', 'P']

FOI_COLUMNS = ["FOI_hits_X", "FOI_hits_Y", "FOI_hits_T",
               "FOI_hits_Z", "FOI_hits_DX", "FOI_hits_DY", "FOI_hits_S"]

ID_COLUMN = "id"

N_STATIONS = 4
FEATURES_PER_STATION = 6
N_FOI_FEATURES = N_STATIONS*FEATURES_PER_STATION
# The value to use for stations with missing hits
# when computing FOI features
EMPTY_FILLER = 1000

# Examples on working with the provided files in different ways

VERSION = "v2"
# hdf is all fine - but it requires unpickling the numpy arrays
# which is not guranteed
def load_train_hdf(path):
    return pd.concat([
        pd.read_hdf(os.path.join(path, "train_part_%i_%s.hdf" % (i, VERSION)))
        for i in (1, 1)], axis=0, ignore_index=True)


def load_data_csv(path, feature_columns= [ID_COLUMN] + SIMPLE_FEATURE_COLUMNS):
    converters = dict(zip(FOI_COLUMNS, repeat(parse_array)))
    types = dict(zip(SIMPLE_FEATURE_COLUMNS, repeat(np.float32)))
    train = pd.concat([
        pd.read_csv(os.path.join(path, "train_part_%i_%s.csv.gz" % (i, VERSION)),
                    usecols= [ID_COLUMN] + feature_columns + TRAIN_COLUMNS,
                       dtype=types,
                    index_col=ID_COLUMN)
        for i in (1, 2)], axis=0, ignore_index=True)
    test = pd.read_csv(os.path.join(path, "test_private_%s_track_1.csv" % VERSION),
                       dtype=types,
                       usecols=[ID_COLUMN] + feature_columns, index_col=ID_COLUMN)
    return train, test


def parse_array(line, dtype=np.float32):
    return np.fromstring(line[1:-1], sep=" ", dtype=dtype)


def load_full_test_csv(path):
    converters = dict(zip(FOI_COLUMNS, repeat(parse_array)))
    types = dict(zip(SIMPLE_FEATURE_COLUMNS, repeat(np.float32)))
    test = pd.read_csv(os.path.join(path, "test_public_%s.csv.gz" % VERSION),
                       index_col="id", converters=converters,
                       dtype=types,
                       usecols=[ID_COLUMN]+SIMPLE_FEATURE_COLUMNS+FOI_COLUMNS)
    return test


def find_closest_hit_per_station(row):
    result = np.empty(N_FOI_FEATURES, dtype=np.float32)
    closest_x_per_station = result[0:4]
    closest_y_per_station = result[4:8]
    closest_T_per_station = result[8:12]
    closest_z_per_station = result[12:16]
    closest_dx_per_station = result[16:20]
    closest_dy_per_station = result[20:24]
    
    for station in range(4):
        hits = (row["FOI_hits_S"] == station)
        if not hits.any():
            closest_x_per_station[station] = EMPTY_FILLER
            closest_y_per_station[station] = EMPTY_FILLER
            closest_T_per_station[station] = EMPTY_FILLER
            closest_z_per_station[station] = EMPTY_FILLER
            closest_dx_per_station[station] = EMPTY_FILLER
            closest_dy_per_station[station] = EMPTY_FILLER
        else:
            x_distances_2 = (row["Lextra_X[%i]" % station] - row["FOI_hits_X"][hits])**2
            y_distances_2 = (row["Lextra_Y[%i]" % station] - row["FOI_hits_Y"][hits])**2
            distances_2 = x_distances_2 + y_distances_2
            closest_hit = np.argmin(distances_2)
            closest_x_per_station[station] = x_distances_2[closest_hit]
            closest_y_per_station[station] = y_distances_2[closest_hit]
            closest_T_per_station[station] = row["FOI_hits_T"][hits][closest_hit]
            closest_z_per_station[station] = row["FOI_hits_Z"][hits][closest_hit]
            closest_dx_per_station[station] = row["FOI_hits_DX"][hits][closest_hit]
            closest_dy_per_station[station] = row["FOI_hits_DY"][hits][closest_hit]
            
    return result

# Prepare data

In [44]:
DATA_PATH = "data"

In [64]:
full_train, test = load_data_csv(DATA_PATH)

/home/ihar/.local/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [65]:
# Can be calculated using find_closest_hit_per_station function for test data

closest_hits_features_test = pd.read_csv('private_features.csv', index_col=0, dtype=np.float32)

/home/ihar/.local/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [66]:
test_concat = pd.concat(
    [test.loc[:, SIMPLE_FEATURE_COLUMNS],
     closest_hits_features_test], axis=1)

In [67]:
test_concat.shape

(1452188, 89)

In [47]:
# Can be calculated using find_closest_hit_per_station function for train data
closest_hits_features = pd.read_csv('closest_hits_features.csv', index_col=0, dtype=np.float32)

/home/ihar/.local/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [51]:
train_concat = pd.concat(
    [full_train.loc[:, SIMPLE_FEATURE_COLUMNS],,
     closest_hits_features], axis=1)

In [19]:
ntrain = 5445705
ntest = 1452188

In [68]:
X_train = train_concat.values

In [ ]:
from itertools import chain

X_train_sklearn = []
X_test_sklearn = []
X_emb_sklearn = []

In [69]:
X_train_sklearn = np.array(X_train[:ntrain])
X_test_sklearn = np.array(X_train[ntrain:])
Y_train = full_train.particle_type.values

In [70]:
Y_train = full_train.particle_type.values

In [71]:
weight_train = full_train.weight.swifter.apply(lambda x: x if x > 0 else 0).values
weight_train_original = full_train.weight.values

HBox(children=(IntProgress(value=0, description='Dask Apply', max=48, style=ProgressStyle(description_width='initial')), HTML(value='')))

# Test

In [16]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.utils import class_weight
import scoring
import gc
from imblearn.combine import SMOTETomek


n_folds = 5
from sklearn.metrics.scorer import make_scorer

def metric(y_true, y_pred, sample_weight):
    return scoring.rejection90(y_true, y_pred, sample_weight=sample_weight)
                                        
my_scorer = make_scorer(metric, greater_is_better=True)

def unison_shuffled_copies(a, b, c, d):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p], c[p], d[p]


y_pred = []
y = []
weight_train_original_shuffle = []
test_index = []

# Uncomment to shuffle data

# X_train_shuffle, Y_train_shuffle, weight_train_shuffle, \
#         weight_train_original_shuffle = unison_shuffled_copies(X_train_sklearn, Y_train, weight_train,\
#                                                              weight_train_original)
X_train_shuffle, Y_train_shuffle, weight_train_shuffle, \
        weight_train_original_shuffle    = X_train_sklearn, Y_train, weight_train,\
                                                             weight_train_original
def rmsle_cv(model_class):
    """
    A function to test a model performance using k-fold cross validation
    """
    global y_pred, y, weight_train_original_shuffle, test_index
    
    
    kf = KFold(n_folds, shuffle=True)
    scores = []
    for train_index, test_index in kf.split(X_train_sklearn):
        gc.collect()
        X, y = X_train_shuffle[train_index], Y_train_shuffle[train_index]
        sample_weight = weight_train_shuffle[train_index]
        
        
        class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y),
                                                 y)
   
        X_train, y_train = X.copy(), y.copy()
        model = model_class(class_weights)
        
        if model.__class__.__name__ == 'CatBoostClassifier':
            y_train[y_train == 2] = 0
            
    
        model.fit(X_train, y_train, sample_weight=sample_weight)
        X, y = X_train_shuffle[test_index], Y_train_shuffle[test_index]
        y_pred = model.predict_proba(X)[:, 1]
        y[y == 2] = 0
        score = metric(y, y_pred,  weight_train_original_shuffle[test_index])
        scores.append(score)
        print(score)
  
        
    return(np.array(scores))

In [25]:
import lightgbm as lgb
model_lgb = lambda class_weights: lgb.LGBMClassifier(
    num_leaves=200,
    learning_rate=0.1, n_estimators=1500,
    bagging_fraction = 0.8,
    bagging_freq = 3, feature_fraction = 0.8,
    min_data_in_leaf=800,
    device='gpu',
    # is_unbalance=True,
    metrics=['acc'],
    n_jobs=-1,
    verbose=1,
    bagging_temperature=0.15,
)


In [ ]:
# Check lgbm performance
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
import catboost
import gc

gc.collect()
try:
    model_cat = lambda class_weights: catboost.CatBoostClassifier(task_type='CPU', class_weights=None,
                                            iterations=1300, max_depth=5, thread_count=12, verbose=True,
                                                         l2_leaf_reg=None, 
                                                              # bagging_temperature=0.12,
                                                           border_count=52,
                                                                  boosting_type='Plain',
                                                                  max_ctr_complexity=1,
                                                         )
except:
    pass

score = rmsle_cv(model_cat)
print("CatBoost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [27]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone

class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models, class_weights):
        self.models = models
        self.class_weights = class_weights
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y, sample_weight=None):
        y_new = y.copy()
        self.models_ = [x(self.class_weights) for x, _ in self.models]
        self.weights = [weight for _, weight in self.models]
        
        # Train cloned base models
        for model in self.models_:
            print(model.__class__.__name__)
            if model.__class__.__name__ == 'CatBoostClassifier':
                y_new[y_new == 2] = 0
                print(y_new)
            model.fit(X, y_new,
                      sample_weight=sample_weight)
            if model.__class__.__name__ == 'CatBoostClassifier':
                model.save_model("model_cat_avg.cbm")

        return self
    
    # Now we do the predictions for cloned models and average them
    
    def predict_proba(self, X):
        sum_weight = 0
        for weight in self.weights:
            sum_weight += weight
            
        predictions = np.add(*[
            model.predict_proba(X) * (self.weights[i] / sum_weight) for i, model in enumerate(self.models_)
        ])
        return predictions  

In [28]:
averaged_models = lambda class_weights: AveragingModels(models = ((model_cat, 1), (model_lgb, 1)),
                                                        class_weights=class_weights)

# score = rmsle_cv(averaged_models)
# print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

# Predict

In [ ]:
model_cat = lambda class_weights: catboost.CatBoostClassifier(task_type='GPU', class_weights=None,
                                            iterations=1300, max_depth=5, thread_count=12, verbose=True,
                                                         l2_leaf_reg=None, bagging_temperature=0.12,
                                                         )

averaged_models = lambda class_weights: AveragingModels(models = ((model_cat, 1), (model_lgb, 1)),
                                                        class_weights=class_weights)
model = averaged_models(None)

X, y = X_train_sklearn, Y_train

if model.__class__.__name__ == 'CatBoostClassifier':
    y[y == 2] = 0

sample_weight_train = weight_train
model.fit(X, y, sample_weight=sample_weight_train)
y_pred = model.predict_proba(test_concat.values)[:, 1]


In [73]:
pd.DataFrame(data={"prediction": y_pred}, index=test.index).to_csv(
    "submission30_final.csv", index_label=ID_COLUMN)